In [1]:
from Controller import ScenarioCfController
from Utils.SaveUtils import load_parameters
from Parameters import TrainingParameters
import tensorflow as tf
import numpy as np
import pandas as pd

In [2]:
# %tensorboard --logdir logs/gradient_tape --host localhost --port 8088

In [3]:
parameters = TrainingParameters()
tf.random.set_seed(parameters.dataset_split_seed)
np.random.seed(parameters.dataset_split_seed)
controller = ScenarioCfController(parameters = parameters)


| Running on /job:localhost/replica:0/task:0/device:CPU:0  


parsing log, completed traces ::   0%|          | 0/13087 [00:00<?, ?it/s]


| Preprocessed data saved successfully 


In [4]:
len(controller.dataset.df)

523480

In [5]:
controller.show_model_info()

Model: "scenario_cf_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  832       
_________________________________________________________________
embedding_1 (Embedding)      multiple                  9216      
_________________________________________________________________
lstm (LSTM)                  multiple                  8320      
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8320      
_________________________________________________________________
lstm_2 (LSTM)                multiple                  20608     
_________________________________________________________________
lstm_3 (LSTM)                multiple                  8320      
_________________________________________________________________
sequential (Sequential)      (1, 1, 1)           

In [6]:
controller.train()


| Training records in logs/gradient_tape/20210627-013946 

| Total epochs: 10 

| Total steps: 32720 

| Start epoch 0 

| Evaluation result | Loss [0.5755] | Accuracy [0.4962]  

| Evaluation result | Loss [0.4983] | Accuracy [0.5049]  

| Evaluation result | Loss [0.3935] | Accuracy [0.7760]  

| Evaluation result | Loss [0.3030] | Accuracy [0.8509]  

| Evaluation result | Loss [0.2116] | Accuracy [0.9227]  

| Evaluation result | Loss [0.1259] | Accuracy [0.9541]  

| Evaluation result | Loss [0.0837] | Accuracy [0.9581]  

| Evaluation result | Loss [0.0500] | Accuracy [0.9644]  

| Evaluation result | Loss [0.0421] | Accuracy [0.9801]  

| Evaluation result | Loss [0.0384] | Accuracy [0.9840]  

| Evaluation result | Loss [0.0378] | Accuracy [0.9840]  

| Evaluation result | Loss [0.0375] | Accuracy [0.9813]  

| Evaluation result | Loss [0.0355] | Accuracy [0.9839]  

| Evaluation result | Loss [0.0356] | Accuracy [0.9841]  

| Evaluation result | Loss [0.0355] | Accuracy [0.98


| Evaluation result | Loss [0.0327] | Accuracy [0.9850]  

| Evaluation result | Loss [0.0327] | Accuracy [0.9853]  

| Evaluation result | Loss [0.0326] | Accuracy [0.9853]  

| Evaluation result | Loss [0.0327] | Accuracy [0.9855]  

| Evaluation result | Loss [0.0326] | Accuracy [0.9850]  

| Evaluation result | Loss [0.0325] | Accuracy [0.9852]  

| Evaluation result | Loss [0.0326] | Accuracy [0.9854]  

| Evaluation result | Loss [0.0326] | Accuracy [0.9852]  

| Evaluation result | Loss [0.0327] | Accuracy [0.9854]  

| Evaluation result | Loss [0.0326] | Accuracy [0.9853]  

| Evaluation result | Loss [0.0326] | Accuracy [0.9856]  

| Evaluation result | Loss [0.0326] | Accuracy [0.9850]  

| Evaluation result | Loss [0.0326] | Accuracy [0.9849]  

| Evaluation result | Loss [0.0326] | Accuracy [0.9851]  

| Evaluation result | Loss [0.0326] | Accuracy [0.9850]  

| Evaluation result | Loss [0.0325] | Accuracy [0.9853]  

| Evaluation result | Loss [0.0325] | Accuracy [0.9851]

KeyboardInterrupt: 

In [9]:
###### Save model
controller.save_training_result("classify_valid_trace.ipynb")


| Model saved successfully to: ./SavedModels/BPI2012WithResource_ValidPath_2021-06-27 05:38:17.057241  


In [ ]:
###### Loading test
# controller.load_trained_model("./SavedModels/0.9860_BPI2012WithResource_BaselineLSTMWithResource_2021-06-25 21:03:20.326179")

In [ ]:
from Utils.DiCEHelpers import generate_fake_df, get_trace_with_id, get_longest_trace_row, remove_trail_steps, print_model_prediction_result, remove_tags_for_query_instance

folder_path = "./SavedModels/%s" % (
    "0.8264_BPI2012WithResource_BaselineLSTMWithResource_2021-06-18 06:11:10.009443" #AOW
)

### Initialise controllers
parameters_json = load_parameters(folder_path=folder_path)
parameters = TrainingParameters(**parameters_json)
tf.random.set_seed(parameters.dataset_split_seed)
np.random.seed(parameters.dataset_split_seed)
parameters.load_model_folder_path = folder_path
trainer = TrainingController(parameters = parameters)
trainer.show_model_info()

########### Get example data from trainer ###########
ordered_test_idx = (list(trainer.test_dataset.unbatch().as_numpy_iterator()))
ordered_test_idx.sort()
print_big(len(ordered_test_idx), "Test set length")

########## Get longest delcined trace for testing ##########
declined_df = get_trace_with_id(trainer.dataset.df.iloc[ordered_test_idx], trainer.model.vocab.vocab_to_index('A_DECLINED_COMPLETE'))
longest_declined_trace_row = get_longest_trace_row(declined_df)
longest_declined_trace_row

_, example_activities, _, example_resources, example_amount, _ = trainer.dataset.collate_fn([longest_declined_trace_row.index[0]])

tail_length_to_remove = 45
example_idx_activities, example_idx_resources = remove_trail_steps(example_activities, example_resources, tail_length_to_remove)
example_vocab_activities = trainer.model.vocab.list_of_index_to_vocab_2d(example_idx_activities)[0]
print("========================================Trace after substraction========================================")
print(example_vocab_activities)

##### Get model output
example_activities_input = tf.constant(example_idx_activities)
example_resources_input = tf.constant(example_idx_resources)
example_amount_input = tf.constant(example_amount)

predicted_df = print_model_prediction_result(trainer.model, example_activities_input, example_resources_input, example_amount_input)